# 日付毎の陽性者数を集計

In [1]:
import json

import pandas as pd
import requests

# 東京都_新型コロナ受診相談窓口相談件数 CSVファイル
TEL_CSV_URL = "https://stopcovid19.metro.tokyo.lg.jp/data/130001_tokyo_covid19_combined_telephone_advice_center.csv"
# 東京都_新型コロナウイルス陽性患者発表詳細
PATIENTS_CSV_URL = "https://stopcovid19.metro.tokyo.lg.jp/data/130001_tokyo_covid19_patients.csv"
# 東京都 新型コロナウイルス感染症対策サイト オープンデータ Json
DATA_JSON_URL = "https://raw.githubusercontent.com/tokyo-metropolitan-gov/covid19/development/data/data.json"

### 陽性者のCSVをデータフレーム化

In [2]:
res = requests.get(PATIENTS_CSV_URL)
csv_data = [row.split(",") for row in res.content.decode("utf-8-sig").split("\r\n")[:-1]]
patients_df = pd.DataFrame(csv_data[1:], columns=csv_data[0])

patients_df

,No,全国地方公共団体コード,都道府県名,市区町村名,公表_年月日,曜日,発症_年月日,患者_居住地,患者_年代,患者_性別,患者_属性,患者_状態,患者_症状,患者_渡航歴の有無フラグ,備考,退院済フラグ
0,1,130001,東京都,,2020-01-24,金,,湖北省武漢市,40代,男性,,,,,,1
1,2,130001,東京都,,2020-01-25,土,,湖北省武漢市,30代,女性,,,,,,1
2,3,130001,東京都,,2020-01-30,木,,湖南省長沙市,30代,女性,,,,,,1
3,4,130001,東京都,,2020-02-13,木,,都内,70代,男性,,,,,,""""""
4,5,130001,東京都,,2020-02-14,金,,都内,50代,女性,,,,,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5128,5021,130001,東京都,,2020-05-21,木,,"""""",40代,女性,,,,,,""""""
5129,5013,130001,東京都,,2020-05-21,木,,"""""",50代,女性,,,,,,""""""
5130,5017,130001,東京都,,2020-05-21,木,,"""""",60代,女性,,,,,,""""""
5131,5014,130001,東京都,,2020-05-21,木,,"""""",70代,男性,,,,,,""""""


### 使用する列のみ抽出し、集計用の列を追加

In [3]:
patients_df["人数"] = 1
patients_df = patients_df[["公表_年月日", "人数"]]

patients_df

,公表_年月日,人数
0,2020-01-24,1
1,2020-01-25,1
2,2020-01-30,1
3,2020-02-13,1
4,2020-02-14,1
...,...,...
5128,2020-05-21,1
5129,2020-05-21,1
5130,2020-05-21,1
5131,2020-05-21,1


### 陽性者が出ていない日付が存在する可能性があるため、全ての日付に対して0人のデータを挿入

In [4]:
# 範囲内の全ての日付に対し、0人のデータフレームを作成
dates = pd.date_range(min(patients_df["公表_年月日"]), max(patients_df["公表_年月日"]))

dates_df = pd.DataFrame([(_date.strftime("%Y-%m-%d"), 0) for _date in dates], 
                        columns=["公表_年月日", "人数"])

dates_df

,公表_年月日,人数
0,2020-01-24,0
1,2020-01-25,0
2,2020-01-26,0
3,2020-01-27,0
4,2020-01-28,0
...,...,...
114,2020-05-17,0
115,2020-05-18,0
116,2020-05-19,0
117,2020-05-20,0


In [5]:
# データフレームを連結
patients_df = pd.concat([dates_df, patients_df]).reset_index(drop=True)

patients_df

,公表_年月日,人数
0,2020-01-24,0
1,2020-01-25,0
2,2020-01-26,0
3,2020-01-27,0
4,2020-01-28,0
...,...,...
5247,2020-05-21,1
5248,2020-05-21,1
5249,2020-05-21,1
5250,2020-05-21,1


In [6]:
patients_df

,公表_年月日,人数
0,2020-01-24,0
1,2020-01-25,0
2,2020-01-26,0
3,2020-01-27,0
4,2020-01-28,0
...,...,...
5247,2020-05-21,1
5248,2020-05-21,1
5249,2020-05-21,1
5250,2020-05-21,1


### 日付毎に陽性者数を集計

In [7]:
grouped_df = patients_df.groupby(["公表_年月日"]).sum()

grouped_df

,人数
公表_年月日,
2020-01-24,1
2020-01-25,1
2020-01-26,0
2020-01-27,0
2020-01-28,0
...,...
2020-05-17,5
2020-05-18,10
2020-05-19,5
